In [1]:
import numpy as np

from subgraph_matching import (
    YaleStarCatalog,
    StarDatabase,
    PairDatabase,
    SubgraphIsomorphismBasedMatching,
    equatorial2vec,
    vec2equatorial,
)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
seed = 1852
np_random = np.random.default_rng(seed)
log_dir = "./log/"
cash_dir = f"{log_dir}/cash/"
#
epsilon = np.deg2rad(0.1)
theta_FOV = np.deg2rad(80)
theta_min = epsilon
theta_max = 2 * np.arctan(np.sqrt(2) * np.tan(theta_FOV / 2))
M_lim_max = 5.5

In [3]:
### Database ###
# Catalog
catalog = YaleStarCatalog(log_dir=cash_dir)
df_D_C = catalog.get_df()
s_vec = equatorial2vec(catalog.get_RA(), catalog.get_DE())
df_D_C.loc[:, ("s_X", "s_Y", "s_Z")] = s_vec
# StarDB
D_DB = StarDatabase(df_D_C)
D_DB.filtering_by_visual_magnitude(M_lim_max)
D_DB.filtering_by_multiple_stars(theta_min)
D_DB_HR = D_DB.get_HR()
# PairDB
P_DB = PairDatabase(D_DB.get_df(), log_dir=cash_dir)
P_DB.create_catalog(theta_max)
# matching
matching = SubgraphIsomorphismBasedMatching(D_DB, P_DB, epsilon)


In [4]:
### Stars (Summer Triangle) ###
alpha_Deneb = (20 + 41 / 60 + 25.91514 / 3600) * np.pi / 12
alpha_Altair = (19 + 50 / 60 + 46.99855 / 3600) * np.pi / 12
alpha_Vega = (18 + 36 / 60 + 56.33635 / 3600) * np.pi / 12
alpha_Alpheratz = (0 + 8 / 60 + 23.25988 / 3600) * np.pi / 12
alpha_Markab = (23 + 4 / 60 + 45.65345 / 3600) * np.pi / 12

delta_Deneb = (45 + 16 / 60 + 49.2197 / 3600) * np.pi / 180
delta_Altair = (8 + 52 / 60 + 5.9563 / 3600) * np.pi / 180
delta_Vega = (38 + 47 / 60 + 1.28 / 3600) * np.pi / 180
delta_Alpheratz = (29 + 5 / 60 + 25.552 / 3600) * np.pi / 180
delta_Markab = (15 + 12 / 60 + 18.9617 / 3600) * np.pi / 180

s_Deneb = equatorial2vec(alpha_Deneb, delta_Deneb)
s_Altair = equatorial2vec(alpha_Altair, delta_Altair)
s_Vega = equatorial2vec(alpha_Vega, delta_Vega)
s_Alpheratz = equatorial2vec(alpha_Alpheratz, delta_Alpheratz)
s_Markab = equatorial2vec(alpha_Markab, delta_Markab)

In [5]:
# Example of a failed match
candi_IDs, obs_IDs, info = matching.match_stars([s_Alpheratz, s_Altair])
print(info)

Matching number of [0] : 2741
Matching number of [0, 1] : 5798



In [6]:
# Example of successful matching
candi_IDs, obs_IDs, info = matching.match_stars([s_Deneb, s_Altair, s_Vega, s_Alpheratz, s_Markab])
print(info)

Matching number of [2] : 2741
Matching number of [2, 4] : 5761
Matching number of [2, 4, 1] : 39
Matching number of [2, 4, 1, 3] : 1



In [29]:
candi_HRs = [D_DB.get_HR()[candi_ID] for candi_ID in candi_IDs[-1]]
catalog_df = catalog.get_full_df()
matching_df = catalog_df[catalog_df["HR"].isin(candi_HRs)]
matching_df[["HR", "Name", "RAh [h]", "RAm [min]", "RAs [s]", "DE-", "DEd [deg]", "DEm [arcmin]", "DEs [arcsec]"]]

,HR,Name,RAh [h],RAm [min],RAs [s],DE-,DEd [deg],DEm [arcmin],DEs [arcsec]
14,15,21Alp And,0.0,8.0,23.3,+,29.0,5.0,26.0
7000,7001,3Alp Lyr,18.0,36.0,56.3,+,38.0,47.0,1.0
7556,7557,53Alp Aql,19.0,50.0,47.0,+,8.0,52.0,6.0
8780,8781,54Alp Peg,23.0,4.0,45.7,+,15.0,12.0,19.0
